<a href="https://colab.research.google.com/github/vichruth/CIFAR-10/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
# Load CIFAR-10 data
def load_batch(file_path):
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
    images = batch[b'data']
    labels = batch[b'labels']
    images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (32, 32, 3)
    return images, np.array(labels)

In [30]:
# Load all training data
x_train, y_train = [], []
for i in range(1, 6):
    images, labels = load_batch(f'/content/cifar-10-batches-py/data_batch_{i}')
    x_train.append(images)
    y_train.append(labels)

x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)
x_train, y_train = [], []
for i in range(1, 6):
    images, labels = load_batch(f'/content/cifar-10-batches-py/data_batch_{i}')
    x_train.append(images)
    y_train.append(labels)

x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)
x_test, y_test = load_batch('/content/cifar-10-batches-py/test_batch')

# Normalize pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0


In [31]:
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [32]:
# Improved CNN Model
model = keras.Sequential([
    data_augmentation,

    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),  # Prevent overfitting
    layers.Dense(10, activation='softmax')
])

# Compile the model with SGD + momentum
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Learning rate scheduler
lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * 0.95 ** epoch)

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[lr_scheduler])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.3224 - loss: 2.1566 - val_accuracy: 0.3919 - val_loss: 1.9264 - learning_rate: 0.0100
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.4810 - loss: 1.4506 - val_accuracy: 0.4712 - val_loss: 1.4924 - learning_rate: 0.0095
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.5210 - loss: 1.3355 - val_accuracy: 0.5637 - val_loss: 1.3625 - learning_rate: 0.0090
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.5611 - loss: 1.2380 - val_accuracy: 0.5978 - val_loss: 1.1594 - learning_rate: 0.0086
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.5921 - loss: 1.1512 - val_accuracy: 0.6280 - val_loss: 1.0667 - learning_rate: 0.0081
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.6204 - loss: 1.0814 - val_accuracy: 0.6304 - val_loss: 1.0651 - learning_rate: 0.0077
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.6366 - los